In [9]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
import pandas as pd
from typing import Dict
from collections import defaultdict
from ephysvibe.trials.spikes import firing_rate,sp_constants,plot_raster
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.structures.trials_data import TrialsData
from ephysvibe.analysis import raw_ch
from ephysvibe.task import task_constants,def_task
import os, glob

## Load data

In [10]:
directory="/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip"

In [11]:
bhv_path = os.path.normpath(str(directory) + "/*.csv")
bhv_path = glob.glob(bhv_path, recursive=True)

In [12]:
bhv_path

['/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-01-20_11-00-24_Riesling_lip_e1_r1_neuron_12rf_coordinates.csv',
 '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-01-23_10-10-27_Riesling_lip_e1_r1_mua_4rf_coordinates.csv',
 '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-01-26_10-45-42_Riesling_lip_e1_r1_neuron_4rf_coordinates.csv',
 '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-02-08_10-39-41_Riesling_lip_e1_r1_mua_6rf_coordinates.csv',
 '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-02-09_10-42-54_Riesling_lip_e1_r1_mua_1rf_coordinates.csv',
 '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-02-24_10-43-44_Riesling_lip_e1_r1_neuron_2rf_coordinates.csv',
 '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-02-27_10-50-03_Riesli

In [13]:


for n_path in bhv_path[:2]:
    rf = pd.read_csv(filepath)
    no_nan_rf = rf[~rf.isnull().any(axis=1)]

In [1]:
filepath = "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/plots2/detect_rf/lip/2023-01-20_11-00-24_Riesling_lip_e1_r1_neuron_12rf_coordinates.csv"

In [3]:
rf = pd.read_csv(filepath)
no_nan_rf = rf[~rf.isnull().any(axis=1)]

## number of ipsi vs number of contra

,i_array,neuron_type,i_neuron,event,rad,ang,fr_max,ang_max,depth,date
10,5,neuron,4,visual,0.832172,0.438803,85.409301,45.0,1250.0,2023-01-20_11-00-24
11,5,neuron,4,motor,0.440295,0.413874,42.775212,45.0,1250.0,2023-01-20_11-00-24
21,10,mua,3,motor,1.000000,5.497787,12.421879,315.0,1400.0,2023-01-20_11-00-24
23,11,mua,4,motor,0.999280,5.497787,15.778554,315.0,1400.0,2023-01-20_11-00-24
